## Scoring the Benchmark for the models

This notebook is for the user bias benchmark metric we use in this problem. We define user bias as the percentage of skips per user on average. For example if a user skips a song 10% of the times on average, he/she will be assigned a skip probability of 10% for all future tracks.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score as AUC

In [2]:
def data_split(data, validation_ratio = 0.15, test_ratio = 0.15):
    """
    Function to split data into train, validation and test based on timestamps
    
    https://stackoverflow.com/questions/42395258/
    
    """
    train_ratio = 1 - validation_ratio - test_ratio 
    data['time_rank'] = data.groupby('userid')['timestamp'].rank()
    data['user_all_songs_count'] = data['userid'].map(data.groupby('userid')['timestamp'].apply(len))
    data['scaled_time_rank'] = data['time_rank']/ data['user_all_songs_count']    
    data.drop(['time_rank', 'user_all_songs_count'], axis=1, inplace=True)    
    train_data = data.loc[data['scaled_time_rank'] <= train_ratio, :]
    validation_data = data.loc[(data['scaled_time_rank'] <= (1 - test_ratio)) & (data['scaled_time_rank'] > train_ratio), :]
    test_data = data.loc[(data['scaled_time_rank'] > (train_ratio + validation_ratio)), :]
    return train_data, validation_data, test_data

In [3]:
pandas_df = pd.read_csv('data_engineered_features500.csv') 
pandas_df = pandas_df.fillna(0)

In [4]:
train_data, validation_data, test_data = data_split(pandas_df, validation_ratio = 0.15, test_ratio = 0.15)

### Using only training data to evaluate the user bias

In [12]:
user_skips = train_data.groupby(['userid'])['skipped'].sum()
user_counts = train_data.groupby(['userid'])['skipped'].count()
user_summary_table = pd.concat([user_skips, user_counts], axis=1)
user_summary_table.columns = ["skips", "total_count"]
user_summary_table.loc[:,'userid'] = user_summary_table.index
user_summary_table.loc[:, 'skip_percent'] = user_summary_table['skips'] / user_summary_table['total_count']


In [13]:
test_data = pd.merge(test_data, user_summary_table,
                          on = ["userid"], how="left")

### Benchmark AUC Score

In [15]:
AUC(np.array(test_data["skipped"]), np.array(test_data["skip_percent"]))

0.75926314622341584